This is a retrospective data analysis of our ESPN Fantasy Football league data at the end of Week 17 for the 2022 season. The code can be re-used for any past season using the same ESPN API.

I used the [Pro Football Reference](https://www.pro-football-reference.com/years/2022/games.htm) for NFL schedules and [Christian Wendt's ESPN Fantasy Football API](https://github.com/cwendt94/espn-api) to extract ESPN league-specific data.

For data visualization, I used the [Altair Data Visualization Library](https://altair-viz.github.io/). Altair was chosen for its interactivity, especially for its ability to display hover data and the compatibility of interactive plots with GitHub Pages.

One question this analysis can answer is: what was the return on investment of a given players with fantasy draft or waiver budget dollars (FAAB).

See src directory for source code.

In [1]:
%cd /Users/jonathancheng/PycharmProjects/espnff/src/

import nfl_schedule as nf
import ff_league_data as ff
from plotting import scatterplot_acquisitions


/Users/jonathancheng/PycharmProjects/espnff/src


## Get NFL Schedule

In [2]:
year_of_interest=2022

path = r'/Users/jonathancheng/PycharmProjects/espnff/data'

league_id = 1094090
year = year_of_interest

swid = "{F191FB8C-DB2D-4D24-91FB-8CDB2DED249D}"
s2='AECJMQHsUHB0FTXdZkw93uY7GRbX8BPnm93Ye6AwvwrMsrZFGg1Lbmi07SWVov2ioN8zGMFDzZiiDSeQCa7WQHaGivGnMfGWLjmfGwkOeLXb5baD1sltp%2B%2BIfHAtl98TpmHgB16ZpGn6g3Bm5vLEA7yDC6HkbD3LSp0E2rGB7hKziLMvZ7mT6ONJFRe8Xp3ApYWSvxPr9cz0pJiI%2FF0blsZ8hyATDJMEyaQ2O%2FypcsViORr6hqYTmXHPuPKnMBfvYC8LQqi1exGw3vnyg6ptsB2Y'

espn_s2 = s2


In [3]:
df_proteam_schedule = nf.get_nfl_schedule(year_of_interest)
season_start_date = nf.get_season_start_date(df_proteam_schedule)

## Generate League object

In [4]:
league = ff.fetch_espn_api(league_id, year, espn_s2, swid)
activity_ls = league.recent_activity(1000000)
wk_ls = ff.get_weeks(league)

## Get Acquisitions Data

In [5]:
# fetch league data, wrangle into acquisitions DataFrame

acq_data_flat_ls = ff.get_acq_ls(activity_ls)

df_acq = ff.build_df_acq(acq_data_flat_ls)

## Get Draft Data

In [6]:
df_draft,drafted_players = ff.build_df_draft(league)

## Get total points of rostered players Dataframe

In [7]:
df_rostered = ff.build_df_rostered(league)

## Get total points of free agent players Dataframe

In [8]:
df_FA = ff.build_df_FA(league)

In [9]:
# Generate all player stats dataframe, including all Free Agents
df_player_stats = ff.build_df_player_stats(df_rostered,df_FA)

In [10]:
df_draft_stats = ff.build_df_draft_stats(df_draft,df_player_stats)
df_acq_stats = ff.build_df_acq_stats(df_acq,df_player_stats)
df_acq_final = ff.build_df_acq_final(season_start_date, df_draft_stats, df_acq_stats, drafted_players)

## Get player_box_scores from fantasy season

In [11]:
df_player_box_scores = ff.build_df_player_box_scores(league, wk_ls)

## Construct df_stints

In [12]:
df_stints=ff.build_df_stints(df_acq_final, df_proteam_schedule, df_player_stats, drafted_players)

In [13]:

df_stints['Total points per stint'] = df_stints.apply(lambda x: ff.get_total_pts_per_player(x['Player'], x['Stint (wks)'], df_player_box_scores),axis=1).fillna(0)

In [14]:
df_stints.head()

,Stint_id,Player,Team,ProTeam,Added,Bid Amount ($),Dropped,Stint (wks),Position,Drafted,Total points per stint
0,0,49ers D/ST,Big Joshy Style,SF,2022-09-07,1,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...",D/ST,True,140.0
1,1,A.J. Brown,Ice City USA,PHI,2022-09-07,34,NaT,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",WR,True,244.1
2,2,AJ Dillon,The Genaissance,GB,2022-09-07,4,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",RB,True,150.3
3,3,Aaron Jones,Kirk-life Balance,GB,2022-09-07,42,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",RB,True,212.8
4,4,Aaron Rodgers,Frankel's Cankles,GB,2022-09-07,3,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",QB,True,226.6


Running Backs in the Draft

In [15]:
plot_title,chart = scatterplot_acquisitions(df_stints, position='RB', acq_by_draft=True)
chart

alt.Chart(...)

Wide Receivers in the Draft

In [26]:
df_stints

,Stint_id,Player,Team,ProTeam,Added,Bid Amount ($),Dropped,Stint (wks),Position,Drafted,Total points per stint
0,0,49ers D/ST,Big Joshy Style,SF,2022-09-07,1,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...",D/ST,True,140.0
1,1,A.J. Brown,Ice City USA,PHI,2022-09-07,34,NaT,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",WR,True,244.1
2,2,AJ Dillon,The Genaissance,GB,2022-09-07,4,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",RB,True,150.3
3,3,Aaron Jones,Kirk-life Balance,GB,2022-09-07,42,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",RB,True,212.8
4,4,Aaron Rodgers,Frankel's Cankles,GB,2022-09-07,3,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...",QB,True,226.6
...,...,...,...,...,...,...,...,...,...,...,...
403,403,Younghoe Koo,Door City,ATL,2022-12-29,1,NaT,"[17, 18]",K,True,8.0
404,404,Zach Ertz,Frankel's Cankles,ARI,2022-09-07,2,2022-11-17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",TE,True,92.1
405,405,Zamir White,Sweetless in Seattle,OAK,2022-11-27,1,2022-12-28,"[12, 13, 14, 15, 16]",RB,False,4.7
406,406,Zay Jones,Kittle Me Elmo,JAX,2022-11-30,5,NaT,"[13, 14, 15, 16, 17, 18]",WR,False,57.7


In [34]:
df_stints['Position'].unique().tolist()

['D/ST', 'WR', 'RB', 'QB', 'TE', 'K']

In [36]:
tuple(i for i in df_stints['Position'].unique().tolist())


('D/ST', 'WR', 'RB', 'QB', 'TE', 'K')

In [63]:
import altair as alt
import pandas as pd


def scatterplot_acquisitions(df_stints, select_acq_method=[True], select_positions=df_stints['Position'].unique().tolist()):
    
    g = df_stints.groupby(by=["Drafted", "Position"])

    df = pd.concat([g.get_group((acq_by_draft,position)) 
           for acq_by_draft in select_acq_method
           for position in select_positions],axis=0)

    if select_acq_method[0]:
        status = "Draft"
    else:
        status = "Waiver"

    positions = ', '.join(select_positions)

    plot_title = f"Position: {positions} , Acquired by: {status}"
    selection = alt.selection_multi(fields=["Team"], bind="legend")

    color = alt.condition(
        selection,
        alt.Color(
            "Team:N",
            scale=alt.Scale(scheme="tableau20"),
        ),
        alt.value("lightgray"),
    )

    chart = (
        alt.Chart(df)
        .mark_circle(size=40)
        .encode(
            alt.X("Bid Amount ($)", axis=alt.Axis(grid=False)),
            alt.Y("Total points per stint", axis=alt.Axis(grid=False)),
            color=color,
            opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
            tooltip=["Player", "Team", "Bid Amount ($)", "Total points per stint"],
        )
        .add_selection(selection)
        .properties(width=450, height=450, title=plot_title)
        .configure_axis(labelFontSize=18, titleFontSize=18)
        .configure_title(fontSize=20)
        .configure_legend(labelFontSize=14, titleFontSize=14)
    )
    return plot_title, chart


In [67]:
plot_title, chart = scatterplot_acquisitions(df_stints, select_acq_method=[False], select_positions=['WR','RB'])
chart.display()

alt.Chart(...)

In [32]:
list(df_stints.groupby(by=["Drafted", "Position"]).keys())

TypeError: 'list' object is not callable

In [42]:
list(df_stints.groupby(by=["Drafted", "Position"]).groups.keys())

[(False, 'D/ST'),
 (False, 'K'),
 (False, 'QB'),
 (False, 'RB'),
 (False, 'TE'),
 (False, 'WR'),
 (True, 'D/ST'),
 (True, 'K'),
 (True, 'QB'),
 (True, 'RB'),
 (True, 'TE'),
 (True, 'WR')]

In [44]:
acq_by_draft=True
position=['RB','WR']
g = df_stints.groupby(by=["Drafted", "Position"])
# pd.concat(g.get_group(h) for h in )

In [48]:
g.groups.keys()

dict_keys([(False, 'D/ST'), (False, 'K'), (False, 'QB'), (False, 'RB'), (False, 'TE'), (False, 'WR'), (True, 'D/ST'), (True, 'K'), (True, 'QB'), (True, 'RB'), (True, 'TE'), (True, 'WR')])

In [52]:
import pandas as pd
df1 = g.get_group((False,'WR'))

In [53]:
df2 = g.get_group((False,'RB'))

,Stint_id,Player,Team,ProTeam,Added,Bid Amount ($),Dropped,Stint (wks),Position,Drafted,Total points per stint
0,0,49ers D/ST,Big Joshy Style,SF,2022-09-07,1,NaT,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...",D/ST,True,140.0
25,25,Bills D/ST,Kittle Me Elmo,BUF,2022-09-07,2,2022-12-14,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]",D/ST,True,110.0
26,26,Bills D/ST,Sweetless in Seattle,BUF,2022-12-17,4,2022-12-28,"[15, 16]",D/ST,True,9.0
43,43,Browns D/ST,Sweetless in Seattle,CLE,2022-09-07,1,2022-10-05,"[1, 2, 3, 4]",D/ST,True,21.0
44,44,Browns D/ST,Flex Player All Stars,CLE,2022-11-30,2,2022-12-07,[13],D/ST,True,31.0
...,...,...,...,...,...,...,...,...,...,...,...
285,285,Michael Badgley,Door City,DET,2022-12-07,1,2022-12-14,[14],K,False,11.0
286,286,Michael Badgley,Sweetless in Seattle,DET,2022-12-24,2,NaT,"[16, 17, 18]",K,False,17.0
339,339,Riley Patterson,Ice City USA,JAX,2022-09-28,1,2022-10-12,"[4, 5]",K,False,10.0
340,340,Robbie Gould,Kittle Me Elmo,SF,2022-11-10,2,2022-11-16,[10],K,False,10.0


In [56]:
pd.concat([g.get_group((False,position)) for position in df_stints['Position'].unique()],axis=0)


,Stint_id,Player,Team,ProTeam,Added,Bid Amount ($),Dropped,Stint (wks),Position,Drafted,Total points per stint
20,20,Bengals D/ST,Frankel's Cankles,CIN,2022-09-16,1,2022-09-23,[2],D/ST,False,3.0
21,21,Bengals D/ST,Kittle Me Elmo,CIN,2022-10-12,1,2022-10-19,[6],D/ST,False,0.0
22,22,Bengals D/ST,Kirk-life Balance,CIN,2022-10-23,1,2022-11-10,"[7, 8, 9]",D/ST,False,18.0
23,23,Bengals D/ST,Fumble .,CIN,2022-11-16,6,2022-11-24,[11],D/ST,False,0.0
24,24,Bengals D/ST,The Genaissance,CIN,2022-12-21,1,NaT,"[16, 18]",D/ST,False,11.0
...,...,...,...,...,...,...,...,...,...,...,...
285,285,Michael Badgley,Door City,DET,2022-12-07,1,2022-12-14,[14],K,False,11.0
286,286,Michael Badgley,Sweetless in Seattle,DET,2022-12-24,2,NaT,"[16, 17, 18]",K,False,17.0
339,339,Riley Patterson,Ice City USA,JAX,2022-09-28,1,2022-10-12,"[4, 5]",K,False,10.0
340,340,Robbie Gould,Kittle Me Elmo,SF,2022-11-10,2,2022-11-16,[10],K,False,10.0


In [54]:
pd.concat([df1,df2],axis=0)

,Stint_id,Player,Team,ProTeam,Added,Bid Amount ($),Dropped,Stint (wks),Position,Drafted,Total points per stint
7,7,Alec Pierce,The Genaissance,IND,2022-10-23,1,2022-11-02,"[7, 8]",WR,False,13.2
73,73,Christian Watson,Door City,GB,2022-11-16,12,NaT,"[11, 12, 13, 15, 16, 17, 18]",WR,False,77.1
89,89,Curtis Samuel,Kittle Me Elmo,WSH,2022-09-14,3,2022-11-30,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]",WR,False,96.6
93,93,DJ Chark,Door City,DET,2022-10-01,3,2022-10-01,[],WR,False,0.0
94,94,DJ Chark,Door City,DET,2022-10-01,0,2022-10-01,[],WR,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...
381,381,Tyler Allgeier,Door City,ATL,2022-10-02,1,2022-10-15,"[4, 5]",RB,False,15.4
382,382,Tyler Allgeier,Fumble .,ATL,2022-11-02,15,2022-11-24,"[9, 10, 11]",RB,False,21.5
383,383,Tyler Allgeier,Sweetless in Seattle,ATL,2022-12-28,18,NaT,"[17, 18]",RB,False,16.0
405,405,Zamir White,Sweetless in Seattle,OAK,2022-11-27,1,2022-12-28,"[12, 13, 14, 15, 16]",RB,False,4.7


In [55]:
plot_title,chart = scatterplot_acquisitions(df_stints, acq_by_draft=True, position='WR')
chart

alt.Chart(...)

## Waiver Wire Acquisitions

In [17]:
plot_title,chart = scatterplot_acquisitions(df_stints, position='WR', acq_by_draft=False)
chart

alt.Chart(...)

Curtis Samuel was the most valuable waiver wire receiver.

In [24]:
for acq_by_draft in [True,False]:
    for position in df_stints['Position'].unique():
        plot_title,chart = scatterplot_acquisitions(df_stints, position=position, acq_by_draft=acq_by_draft)
        chart.display()


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)